# Module 03b: Advanced Logistic Regression
## From Scratch Implementation & Deep Dive

**🎯 Learning Objectives:**
- Implement Logistic Regression **from scratch**
- Understand **Cross-Entropy Loss** in depth
- Manual calculation **step-by-step** (1 epoch)
- **Gradient check** to verify implementation
- Compare **BGD, SGD, Mini-Batch** for classification
- **Threshold optimization** for imbalanced data
- Implement **One-vs-Rest** multi-class from scratch

**📚 Prerequisites:** Module 03a (Logistic Regression basics)


## 🎭 Analogy for Beginners: The Pass/Fail Exam

### Level 1: Linear vs Logistic

**Think of it this way...** You're a teacher grading exams.

**❌ Linear Regression:**
- Input: Hours studied
- Output: 85, 120, -5 (scores can be >100 or negative!)
- **Problem:** Doesn't make sense for binary outcomes

**✅ Logistic Regression:**
- Input: Hours studied
- Output: 0.85 = **85% probability PASS**
- Decision: If P(Pass) > 0.5 → **PASS** ✅

### Level 2: Why Sigmoid?

**Here's the deal...** Sigmoid is an S-curve that smoothly maps any number to [0,1]:
- Study 0 hours → P(Pass) = 0.01 (almost certain fail)
- Study 5 hours → P(Pass) = 0.5 (50-50 coin flip)
- Study 10 hours → P(Pass) = 0.99 (almost certain pass)

**💡 Key Insight:** Sigmoid squashes linear output into valid probability [0,1]!


In [1]:
# ═══════════════════════════════════════════════════════════════
# 📦 ENVIRONMENT SETUP
# ═══════════════════════════════════════════════════════════════
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Random seed for reproducibility
np.random.seed(42)

print('✅ Libraries loaded!')


✅ Libraries loaded!


## 📊 PART 1: VISUAL INTUITION - Sigmoid Function

**Alright, before we dive into code**, let's see why Sigmoid is special. Understanding the shape gives you intuition for everything that follows.


In [2]:
# ═══════════════════════════════════════════════════════════════
# 📈 SIGMOID VISUALIZATION
# ═══════════════════════════════════════════════════════════════
def sigmoid(z):
    """Sigmoid function: squash to [0, 1]"""
    return 1 / (1 + np.exp(-z))

# Generate z values
z = np.linspace(-10, 10, 200)
sig_z = sigmoid(z)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(z, sig_z, 'b-', linewidth=2, label='σ(z) = 1/(1+e^(-z))')
plt.axhline(y=0.5, color='r', linestyle='--', label='Decision Boundary (0.5)')
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('z (linear combination θᵀx)', fontsize=12)
plt.ylabel('σ(z) - Probability', fontsize=12)
plt.title('🎯 Sigmoid Function: Transforming Linear to Probability', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# Annotate key points
plt.scatter([0], [0.5], color='red', s=100, zorder=5)
plt.annotate('Decision\nBoundary', xy=(0, 0.5), xytext=(2, 0.4),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=10, color='red')

plt.show()

print('💡 Insight:')
print('   z → +∞ : σ(z) → 1 (very confident class 1)')
print('   z = 0  : σ(z) = 0.5 (uncertain - coin flip)')
print('   z → -∞ : σ(z) → 0 (very confident class 0)')


c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)


💡 Insight:
   z → +∞ : σ(z) → 1 (very confident class 1)
   z = 0  : σ(z) = 0.5 (uncertain - coin flip)
   z → -∞ : σ(z) → 0 (very confident class 0)


### 🔍 How to Read the Sigmoid Plot:

**X-axis (z):** Linear combination $z = \theta^T x = \theta_0 + \theta_1 x_1 + ...$

**Y-axis (σ(z)):** Probability output [0, 1]

**When z is very positive (e.g., z=5):**
- $\sigma(z) \approx 1$ → Model is very confident it's class 1

**When z = 0 (the red line):**
- $\sigma(0) = 0.5$ → Decision boundary, model is uncertain

**When z is very negative (e.g., z=-5):**
- $\sigma(z) \approx 0$ → Model is very confident it's class 0

**💡 Why the S-shape?**
- Smooth (differentiable) → enables gradient descent
- Output is a valid probability [0, 1]
- Symmetrical around 0.5


## 🧮 PART 2: FROM SCRATCH IMPLEMENTATION

**Now for the fun part** – let's implement Logistic Regression **completely from scratch**. This is where the real learning happens!


In [3]:
# ═══════════════════════════════════════════════════════════════
# 🏗️ LOGISTIC REGRESSION CLASS (FROM SCRATCH)
# ═══════════════════════════════════════════════════════════════
class LogisticRegressionScratch:
    def __init__(self, learning_rate=0.01, n_epochs=100, batch_size=None, 
                 momentum=0.0, regularization='none', lambda_reg=0.01):
        self.lr = learning_rate
        self.epochs = n_epochs
        self.batch_size = batch_size
        self.momentum = momentum
        self.regularization = regularization  # 'none', 'l2', 'l1'
        self.lambda_reg = lambda_reg
        self.theta = None
        self.loss_history = []
        
    def _sigmoid(self, z):
        """Sigmoid activation"""
        # Clip to avoid overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def _compute_loss(self, X, y, theta):
        """Cross-Entropy Loss with regularization"""
        m = len(y)
        h = self._sigmoid(X @ theta)
        
        # Clip predictions to avoid log(0)
        epsilon = 1e-15
        h = np.clip(h, epsilon, 1 - epsilon)
        
        # Cross-Entropy
        loss = -(1/m) * np.sum(y * np.log(h) + (1-y) * np.log(1-h))
        
        # Regularization (excluding bias θ₀)
        if self.regularization == 'l2':
            loss += (self.lambda_reg / (2*m)) * np.sum(theta[1:]**2)
        elif self.regularization == 'l1':
            loss += (self.lambda_reg / m) * np.sum(np.abs(theta[1:]))
        
        return loss
    
    def fit(self, X, y):
        """Train the model"""
        m, n = X.shape
        
        # Add bias term
        X_b = np.c_[np.ones((m, 1)), X]
        
        # Initialize theta
        self.theta = np.zeros((n + 1, 1))
        velocity = np.zeros_like(self.theta)
        
        y = y.reshape(-1, 1)
        
        # Gradient Descent
        for epoch in range(self.epochs):
            # Shuffle data
            indices = np.random.permutation(m)
            X_b_shuffled = X_b[indices]
            y_shuffled = y[indices]
            
            # Determine batch size
            if self.batch_size is None:
                bs = m  # Batch GD
            else:
                bs = self.batch_size
            
            for i in range(0, m, bs):
                xi = X_b_shuffled[i:i+bs]
                yi = y_shuffled[i:i+bs]
                
                # Forward pass
                h = self._sigmoid(xi @ self.theta)
                
                # Gradient
                gradient = (1/bs) * xi.T @ (h - yi)
                
                # Add regularization gradient
                if self.regularization == 'l2':
                    reg_grad = (self.lambda_reg / bs) * np.r_[[[0]], self.theta[1:]]
                    gradient += reg_grad
                elif self.regularization == 'l1':
                    reg_grad = (self.lambda_reg / bs) * np.r_[[[0]], np.sign(self.theta[1:])]
                    gradient += reg_grad
                
                # Momentum update
                velocity = self.momentum * velocity + self.lr * gradient
                self.theta -= velocity
            
            # Log loss
            loss = self._compute_loss(X_b, y, self.theta)
            self.loss_history.append(loss)
        
        return self
    
    def predict_proba(self, X):
        """Return probabilities"""
        X_b = np.c_[np.ones((len(X), 1)), X]
        return self._sigmoid(X_b @ self.theta)
    
    def predict(self, X, threshold=0.5):
        """Return class predictions"""
        return (self.predict_proba(X) >= threshold).astype(int)

print('✅ LogisticRegressionScratch class created!')
print('   Features: BGD/SGD/Mini-Batch, Momentum, L1/L2 Regularization')


✅ LogisticRegressionScratch class created!
   Features: BGD/SGD/Mini-Batch, Momentum, L1/L2 Regularization


## 🧮 PART 3: MANUAL CALCULATION (1 Epoch Step-by-Step)

**Alright, here's where it gets interesting:** Let's compute **by hand** 1 epoch of Gradient Descent for Logistic Regression. This builds your intuition in a way no amount of reading can!


In [4]:
# ═══════════════════════════════════════════════════════════════
# 📝 MANUAL CALCULATION: 1 EPOCH LOGISTIC REGRESSION
# ═══════════════════════════════════════════════════════════════
print('🧮 MANUAL LOGISTIC REGRESSION CALCULATION')
print('='*60)

# Tiny dataset
X_tiny = np.array([[1], [2], [3]])
y_tiny = np.array([[0], [0], [1]])  # Binary labels
X_tiny_b = np.c_[np.ones((3, 1)), X_tiny]  # Add bias

# Initial theta
theta = np.array([[0], [0]])  # [theta0, theta1]
alpha = 0.1  # learning rate
m = 3

print(f'Data: X = {X_tiny.flatten()}, y = {y_tiny.flatten()}')
print(f'Initial theta: {theta.flatten()}')
print(f'Learning rate: {alpha}')
print()

# Step 1: Linear combination z
z = X_tiny_b @ theta
print('STEP 1: Compute z = X @ theta')
print(f'  z = {z.flatten()}')
print()

# Step 2: Sigmoid
def sigmoid_manual(z):
    return 1 / (1 + np.exp(-z))

h = sigmoid_manual(z)
print('STEP 2: Apply sigmoid h = σ(z)')
print(f'  h = {h.flatten()}')
print('  (These are predicted probabilities)')
print()

# Step 3: Errors
errors = h - y_tiny
print('STEP 3: Compute errors (h - y)')
print(f'  errors = {errors.flatten()}')
print()

# Step 4: Gradient
gradient = (1/m) * X_tiny_b.T @ errors
print('STEP 4: Compute gradient = (1/m) * X.T @ errors')
print(f'  gradient = {gradient.flatten()}')
print()

# Step 5: Update
theta_new = theta - alpha * gradient
print('STEP 5: Update theta = theta - alpha * gradient')
print(f'  theta_new = {theta_new.flatten()}')
print()

# Step 6: Compute loss
epsilon = 1e-15
h_clip = np.clip(h, epsilon, 1-epsilon)
loss = -(1/m) * np.sum(y_tiny * np.log(h_clip) + (1-y_tiny) * np.log(1-h_clip))
print('STEP 6: Compute Cross-Entropy Loss')
print(f'  loss = {loss:.4f}')
print()

print('='*60)
print('💡 After 1 epoch, theta changed from', theta.flatten(), 'to', theta_new.flatten())
print('   This process repeats hundreds/thousands of times until convergence!')


🧮 MANUAL LOGISTIC REGRESSION CALCULATION
Data: X = [1 2 3], y = [0 0 1]
Initial theta: [0 0]
Learning rate: 0.1

STEP 1: Compute z = X @ theta
  z = [0. 0. 0.]

STEP 2: Apply sigmoid h = σ(z)
  h = [0.5 0.5 0.5]
  (These are predicted probabilities)

STEP 3: Compute errors (h - y)
  errors = [ 0.5  0.5 -0.5]

STEP 4: Compute gradient = (1/m) * X.T @ errors
  gradient = [0.16666667 0.        ]

STEP 5: Update theta = theta - alpha * gradient
  theta_new = [-0.01666667  0.        ]

STEP 6: Compute Cross-Entropy Loss
  loss = 0.6931

💡 After 1 epoch, theta changed from [0 0] to [-0.01666667  0.        ]
   This process repeats hundreds/thousands of times until convergence!


### 🔍 How to Read the Manual Calculation:

**STEP 1: Linear Combination**
- $z = \theta^T x$ = Sum of weighted inputs
- With θ = [0, 0], all z values = 0

**STEP 2: Sigmoid**
- $h = \sigma(z) = \frac{1}{1+e^{-z}}$
- With z = 0 for all → h = 0.5 (uncertain)

**STEP 3: Errors**
- Error = Predicted - Actual
- Negative error → model under-predicted
- Positive error → model over-predicted

**STEP 4: Gradient**
- Direction for updating θ
- Negative gradient → increase θ
- Positive gradient → decrease θ

**STEP 5: Update**
- θ_new = θ - α × gradient
- θ moves in the direction that minimizes loss

**STEP 6: Loss**
- Cross-Entropy = measure of error
- Goal: minimize loss → better predictions


## 🔍 PART 4: GRADIENT CHECK

**Now here's how we verify** our implementation is **CORRECT** – by comparing:
- **Analytical gradient** (what we implemented)
- **Numerical gradient** (finite difference approximation)

If they match, we know our math is spot on!

In [5]:
# ═══════════════════════════════════════════════════════════════
# 🔬 GRADIENT CHECK
# ═══════════════════════════════════════════════════════════════
def sigmoid_gc(z):
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def compute_loss_gc(theta, X, y):
    """Cross-Entropy Loss"""
    m = len(y)
    h = sigmoid_gc(X @ theta)
    epsilon = 1e-15
    h = np.clip(h, epsilon, 1-epsilon)
    return -(1/m) * np.sum(y * np.log(h) + (1-y) * np.log(1-h))

def numerical_gradient(theta, X, y, epsilon=1e-7):
    """Numerical gradient using finite difference"""
    num_grad = np.zeros_like(theta)
    for i in range(len(theta)):
        theta_plus = theta.copy()
        theta_minus = theta.copy()
        theta_plus[i] += epsilon
        theta_minus[i] -= epsilon
        
        loss_plus = compute_loss_gc(theta_plus, X, y)
        loss_minus = compute_loss_gc(theta_minus, X, y)
        
        num_grad[i] = (loss_plus - loss_minus) / (2 * epsilon)
    return num_grad

def analytical_gradient(theta, X, y):
    """Analytical gradient (formula)"""
    m = len(y)
    h = sigmoid_gc(X @ theta)
    return (1/m) * X.T @ (h - y)

# Test with sample data
X_test = np.c_[np.ones((5, 1)), np.random.randn(5, 2)]
y_test = np.random.randint(0, 2, (5, 1))
theta_test = np.random.randn(3, 1)

num_grad = numerical_gradient(theta_test, X_test, y_test)
ana_grad = analytical_gradient(theta_test, X_test, y_test)

print('🔍 GRADIENT CHECK')
print('='*50)
print(f'Numerical Gradient:  {num_grad.flatten()}')
print(f'Analytical Gradient: {ana_grad.flatten()}')
print(f'Difference: {np.linalg.norm(num_grad - ana_grad):.2e}')
print('='*50)

if np.linalg.norm(num_grad - ana_grad) < 1e-5:
    print('✅ Gradient implementation is CORRECT!')
else:
    print('❌ WARNING: Gradient mismatch - check implementation!')


🔍 GRADIENT CHECK
Numerical Gradient:  [-0.19413759 -0.04552124 -0.25474762]
Analytical Gradient: [-0.19413759 -0.04552124 -0.25474762]
Difference: 4.49e-10
✅ Gradient implementation is CORRECT!


## 📊 PART 5: OPTIMIZER COMPARISON

**Let's compare** **Batch GD**, **SGD**, **Mini-Batch**, and **Momentum** for Logistic Regression. Each has its sweet spot!


In [6]:
# ═══════════════════════════════════════════════════════════════
# 🏁 OPTIMIZER COMPARISON
# ═══════════════════════════════════════════════════════════════
# Generate dataset
X, y = make_classification(n_samples=500, n_features=2, n_informative=2, 
                          n_redundant=0, n_clusters_per_class=1, 
                          random_state=42, flip_y=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'Dataset: {X_train.shape[0]} train, {X_test.shape[0]} test')
print()

# Train different optimizers
models = [
    ('Batch GD', LogisticRegressionScratch(learning_rate=0.1, n_epochs=50, batch_size=None)),
    ('SGD', LogisticRegressionScratch(learning_rate=0.1, n_epochs=50, batch_size=1)),
    ('Mini-Batch', LogisticRegressionScratch(learning_rate=0.1, n_epochs=50, batch_size=32)),
    ('Momentum', LogisticRegressionScratch(learning_rate=0.1, n_epochs=50, batch_size=32, momentum=0.9)),
]

# Plot learning curves
plt.figure(figsize=(12, 5))

for name, model in models:
    model.fit(X_train, y_train)
    plt.plot(model.loss_history, label=name, linewidth=2)
    
    # Evaluate
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'{name:15} - Test Accuracy: {acc:.4f}')

plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Cross-Entropy Loss', fontsize=12)
plt.title('🏁 Optimizer Comparison: Logistic Regression', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


Dataset: 350 train, 150 test

Batch GD        - Test Accuracy: 0.8267
SGD             - Test Accuracy: 0.8333
Mini-Batch      - Test Accuracy: 0.8333
Momentum        - Test Accuracy: 0.8333


c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 127937 (\N{CHEQUERED FLAG}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)


### 🔍 How to Read the Optimizer Comparison:

**Think of it this way...**

**Batch GD:**
- Loss curve is **smoothest**
- Convergence is **stable** but **slow**
- Use for: Small datasets (<10K samples)

**SGD:**
- Loss curve is **noisy/zigzag**
- Updates are **fast** but **unstable**
- Use for: Very large datasets (>1M samples)

**Mini-Batch:**
- Best balance between speed and stability
- **The default choice in practice**
- Typical batch_size: 16, 32, 64

**Momentum:**
- **Faster convergence** than pother mini-batch
- Smooths out oscillations (like a ball rolling downhill)
- Typical momentum: 0.9


**💡 Production Tip:** Start with Mini-Batch + Momentum (batch_size=32, momentum=0.9)

## ⚖️ PART 6: THRESHOLD OPTIMIZATION

**Here's the thing:** Default threshold = 0.5, but it's **not always optimal!**

Let's find the best threshold for different metrics.


In [7]:
# ═══════════════════════════════════════════════════════════════
# ⚖️ THRESHOLD OPTIMIZATION
# ═══════════════════════════════════════════════════════════════
from sklearn.metrics import precision_recall_curve, f1_score, roc_curve, auc

# Train model
model_thresh = LogisticRegressionScratch(learning_rate=0.1, n_epochs=100, batch_size=32, momentum=0.9)
model_thresh.fit(X_train, y_train)

# Get probabilities
y_proba = model_thresh.predict_proba(X_test).flatten()

# 1. Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)

# Find optimal threshold for F1
f1_scores = 2 * (precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-10)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds_pr[optimal_idx]

plt.figure(figsize=(14, 5))

# Plot 1: Precision-Recall Curve
plt.subplot(1, 2, 1)
plt.plot(recall, precision, 'b-', linewidth=2)
plt.scatter(recall[optimal_idx], precision[optimal_idx], color='red', s=100, 
           label=f'Optimal (thresh={optimal_threshold:.2f})')
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('📊 Precision-Recall Curve', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: F1 vs Threshold
plt.subplot(1, 2, 2)
plt.plot(thresholds_pr, f1_scores, 'g-', linewidth=2)
plt.axvline(x=optimal_threshold, color='r', linestyle='--', 
           label=f'Optimal Threshold: {optimal_threshold:.2f}')
plt.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, label='Default (0.5)')
plt.xlabel('Threshold', fontsize=12)
plt.ylabel('F1 Score', fontsize=12)
plt.title('🎯 F1 Score vs Threshold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare default vs optimal
y_pred_default = model_thresh.predict(X_test, threshold=0.5)
y_pred_optimal = model_thresh.predict(X_test, threshold=optimal_threshold)

print('\n📊 Threshold Comparison:')
print(f'Default (0.5):        F1 = {f1_score(y_test, y_pred_default):.4f}')
print(f'Optimal ({optimal_threshold:.2f}):      F1 = {f1_score(y_test, y_pred_optimal):.4f}')
print(f'\nImprovement: {((f1_score(y_test, y_pred_optimal) / f1_score(y_test, y_pred_default)) - 1) * 100:.2f}%')


C:\Users\test\Downloads\mingw-w64-ucrt-x86_64-gcc-15.2.0-8-any.pkg\ucrt64\bin\ipykernel_36136\796663651.py:46: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\test\Downloads\mingw-w64-ucrt-x86_64-gcc-15.2.0-8-any.pkg\ucrt64\bin\ipykernel_36136\796663651.py:46: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) Arial.
  plt.tight_layout()
c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)
c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)



📊 Threshold Comparison:
Default (0.5):        F1 = 0.8276
Optimal (0.60):      F1 = 0.8286

Improvement: 0.12%


### 🔍 How to Read Threshold Optimization:

**Left Plot (Precision-Recall Curve):**
- **If the red dot is in the upper right:** High precision & recall (great!)
- **If curve hugs the upper right:** Good classifier overall

**Right Plot (F1 vs Threshold):**
- **Peak (red line):** Optimal threshold to maximize F1
- **If optimal << 0.5:** Model prefers lower threshold (more sensitive)
- **If optimal >> 0.5:** Model prefers higher threshold (more specific)

**When to Use Custom Thresholds?**

| Scenario | Threshold Strategy | Reason |
|----------|-------------------|--------|
| **Fraud Detection** | Lower (0.3-0.4) | Prioritize recall (don't miss fraud!) |
| **Spam Filter** | Higher (0.6-0.7) | Prioritize precision (avoid false positives) |
| **Medical Diagnosis** | Lower (0.2-0.3) | Better safe than sorry |
| **Balanced Dataset** | Optimal F1 | Maximize overall performance |


## 🎯 PART 7: MULTI-CLASS CLASSIFICATION (One-vs-Rest)

**Here's the deal:** Logistic Regression is natively **binary**. For multi-class, we use **One-vs-Rest** (OvR):
- Train K binary classifiers (K = number of classes)
- Classifier i: Class i vs All Others
- Prediction: Pick the class with highest probability


It's elegant and it works!

In [8]:
# ═══════════════════════════════════════════════════════════════
# 🎯 ONE-VS-REST MULTI-CLASS
# ═══════════════════════════════════════════════════════════════
class OneVsRestLogistic:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.classifiers = []
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        
        for c in self.classes:
            # Binary problem: class c vs others
            y_binary = (y == c).astype(int)
            
            # Train classifier
            clf = LogisticRegressionScratch(**self.kwargs)
            clf.fit(X, y_binary)
            self.classifiers.append(clf)
        
        return self
    
    def predict_proba(self, X):
        """Return probabilities for each class"""
        probas = []
        for clf in self.classifiers:
            probas.append(clf.predict_proba(X))
        
        return np.hstack(probas)
    
    def predict(self, X):
        """Return class with highest probability"""
        probas = self.predict_proba(X)
        return self.classes[np.argmax(probas, axis=1)]

# Test with 3-class dataset
X_multi, y_multi = make_classification(n_samples=600, n_features=2, n_informative=2, 
                                      n_redundant=0, n_classes=3, n_clusters_per_class=1,
                                      random_state=42)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_multi, y_multi, test_size=0.3, random_state=42)

print(f'Multi-class dataset: {len(np.unique(y_multi))} classes')
print(f'Train: {X_train_m.shape[0]}, Test: {X_test_m.shape[0]}')
print()

# Train One-vs-Rest
ovr = OneVsRestLogistic(learning_rate=0.1, n_epochs=100, batch_size=32, momentum=0.9)
ovr.fit(X_train_m, y_train_m)

# Predict
y_pred_m = ovr.predict(X_test_m)
acc = accuracy_score(y_test_m, y_pred_m)

print(f'✅ One-vs-Rest Accuracy: {acc:.4f}')
print()

# Confusion Matrix
cm = confusion_matrix(y_test_m, y_pred_m)
print('Confusion Matrix:')
print(cm)

# Visualize decision boundaries
plt.figure(figsize=(10, 6))

# Create mesh
h = 0.02
x_min, x_max = X_multi[:, 0].min() - 1, X_multi[:, 0].max() + 1
y_min, y_max = X_multi[:, 1].min() - 1, X_multi[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predict on mesh
Z = ovr.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.Paired)
plt.scatter(X_multi[:, 0], X_multi[:, 1], c=y_multi, edgecolors='k', cmap=plt.cm.Paired, s=50)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('🎯 One-vs-Rest Multi-Class Decision Boundaries', fontsize=14)
plt.show()


Multi-class dataset: 3 classes
Train: 420, Test: 180

✅ One-vs-Rest Accuracy: 0.9111

Confusion Matrix:
[[54  3  0]
 [ 2 62  4]
 [ 1  6 48]]


c:\Users\test\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\pylabtools.py:152: UserWarning: Glyph 127919 (\N{DIRECT HIT}) missing from font(s) Arial.
  fig.canvas.print_figure(bytes_io, **kw)


## 📝 SUMMARY: Module 03b

### 🎓 What We Learned:

**Think of it this way...** We went from theory to implementation, covering everything you need to build Logistic Regression from the ground up.

| # | Topic | Key Insight |
|---|-------|-------------|
| 1 | **Sigmoid Function** | Squashes linear output to [0,1] probability |
| 2 | **From Scratch** | Implemented Logistic with BGD/SGD/Momentum |
| 3 | **Manual Calculation** | 1 epoch step-by-step (6 steps) |
| 4 | **Gradient Check** | Verified implementation (numerical vs analytical) |
| 5 | **Optimizer Comparison** | Mini-Batch + Momentum = best default |
| 6 | **Threshold Optimization** | Tune for imbalanced data |
| 7 | **Multi-Class (OvR)** | Extended binary to K classes |

---

### 🔑 Key Takeaways:

**1. Cross-Entropy Loss:**
$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log h_\theta(x^{(i)}) + (1-y^{(i)}) \log (1 - h_\theta(x^{(i)})) \right]$$

**2. Gradient Update:**
$$\theta := \theta - \alpha \cdot \frac{1}{m} X^T (h - y)$$

**3. Decision Rule:**
- Predict 1 if $h_\theta(x) \geq$ threshold
- Predict 0 otherwise
- **Default:** threshold = 0.5
- **Optimal:** Find via Precision-Recall curve

**4. Regularization:**
- L2 (Ridge): $J(\theta) + \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2$
- L1 (Lasso): $J(\theta) + \frac{\lambda}{m} \sum_{j=1}^{n} |\theta_j|$

---

### 🚀 Next Steps:

1. **Module 04:** Softmax Regression (multi-class native)
2. **Module 06:** Newton's Method (faster convergence)
3. **Module 10:** SVM (margin-based classifier)

---

### 💡 Production Checklist:

- [ ] Feature scaling (StandardScaler)
- [ ] Regularization (L1/L2)
- [ ] Threshold tuning (if imbalanced)
- [ ] Cross-validation (K-Fold)
- [ ] Monitoring (track loss, accuracy over time)

---

**Congratulations! Module 03b COMPLETE!** 🎉

You now understand Logistic Regression at a deep level - from the math to the implementation. That's powerful!
